In [1]:
#HyprParams
inputdim = 10

In [11]:
#Sequential 모델은 레이어를 선형적으로 쌓는다.
from numpy import array
from keras  import Sequential
from keras.layers import Dense, Activation, LSTM

x = array([[1,2,3], [2,3,4], [3,4,5], [4,5,6], [5,6,7], [6,7,8], [7,8,9] , [8,9,10], [9,10,11], [10,11,12], [20,30,40], [40,50,60]])
y = array([4,5,6,7,8,9,10,11,12,13, 50, 70])


In [14]:
print(x.shape)
print(y.shape)

(12, 3)
(12,)


In [15]:
x = x.reshape((x.shape[0], x.shape[1], 1))

In [19]:
print(x.shape)

(12, 3, 1)


In [24]:
model = Sequential()
model.add(LSTM(20, activation = 'relu', input_shape=(3,1)))
model.add(Dense(5))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, mode='auto')
model.fit(x, y, epochs=1000, batch_size=1, verbose=2, callbacks=[early_stopping])
x_input = array([25,35,45])
x_input = x_input.reshape((1,3,1))

yhat = model.predict(x_input)
print(yhat)

Epoch 1/1000
 - 0s - loss: 818.6042
Epoch 2/1000
 - 0s - loss: 797.1993
Epoch 3/1000
 - 0s - loss: 780.8500
Epoch 4/1000
 - 0s - loss: 764.9880
Epoch 5/1000
 - 0s - loss: 753.1790
Epoch 6/1000
 - 0s - loss: 740.7639
Epoch 7/1000
 - 0s - loss: 722.7836
Epoch 8/1000
 - 0s - loss: 705.7840
Epoch 9/1000
 - 0s - loss: 679.2733
Epoch 10/1000
 - 0s - loss: 652.7916
Epoch 11/1000
 - 0s - loss: 555.8636
Epoch 12/1000
 - 0s - loss: 416.4674
Epoch 13/1000
 - 0s - loss: 214.7184
Epoch 14/1000
 - 0s - loss: 36.9072
Epoch 15/1000
 - 0s - loss: 20.2605
Epoch 16/1000
 - 0s - loss: 7.6340
Epoch 17/1000
 - 0s - loss: 5.3260
Epoch 18/1000
 - 0s - loss: 5.2264
Epoch 19/1000
 - 0s - loss: 5.4181
Epoch 20/1000
 - 0s - loss: 5.3306
Epoch 21/1000
 - 0s - loss: 5.5058
Epoch 22/1000
 - 0s - loss: 5.1233
Epoch 23/1000
 - 0s - loss: 5.1658
Epoch 24/1000
 - 0s - loss: 5.2070
Epoch 25/1000
 - 0s - loss: 4.9830
Epoch 26/1000
 - 0s - loss: 4.8881
Epoch 27/1000
 - 0s - loss: 5.0094
Epoch 28/1000
 - 0s - loss: 5.0112
E

In [ ]:
#윤진씨 코드

import pandas as pd
import os
import datetime
import numpy as np

#new_lst=[] # 시간별 호가창의 상태가 저장된 리스트

rate_lst=[]
def func1(): #file 읽어와서 날짜와 체결가 데이터만 저장한 dataframe으로 변환
    in_file = None
    in_file=open("tickData.txt","r") #file open

    in_str=in_file.read() #file read
    lst=in_str.split(')') # split
    new_lst=[]
    for el in lst :
        new_lst.append(el.lstrip(', ('))
    new_lst=list(filter(None,new_lst))
    #########################################리스트에 생성##################3
    time_lst=[]
    price_lst=[]
    for el in new_lst:
        lst2=el.split(',')
        #time=datetime.datetime.strptime(lst2[1].lstrip(),'%H%M%S')
        time=lst2[1].lstrip()
        time=datetime.datetime.now().strftime("%Y%m%d")+time
        time_lst.append(time) #시간리스트에 추가
        price_lst.append(int(lst2[4])) #체결가 리스트에 추가

    in_file.close()
    # dataframe으로 변환 ('시간'열 과 '체결가' 열 2개의 열로 이루어진 dataframe)
    time_lst=pd.to_datetime(time_lst)
    raw_data = {'시간': time_lst, '체결가': price_lst}
    #raw_data = {'체결가': price_lst}
    data = pd.DataFrame(raw_data)
    #data =pd.DataFrame(raw_data, index= time_lst)
    #print(data)
    return data

def aftertimeprice(n,data): #n분 후
    after_price_lst=[]
    for i in range(len(data)) :
        is_time=data['시간']==data.loc[i]['시간']+datetime.timedelta(minutes=n) #각 행에대해서 n분 후의 가격이 존재하면 리스트에 추가
        temp=data[is_time].reset_index(drop=True)
        if(temp.empty==False) :
            after_price_lst.append(temp.loc[0]['체결가'])
        else: #1분후 값이 존재하지 않는 부분 n분 5초후 or  nan으로 채움
            is_time = data['시간'] == data.loc[i]['시간'] + datetime.timedelta(minutes=n,seconds=5)
            temp = data[is_time].reset_index(drop=True)
            if (temp.empty == False):
                after_price_lst.append(temp.loc[0]['체결가'])
            else: #그래도 비었을 경우 nan값
                after_price_lst.append(np.nan)
    #print(len(after_price_lst))
    line=str(n)+"분후"
    data[line]=after_price_lst #n분후 가격 저장한 열추가

    for i in range(len(data)):
        if (data.loc[i][line] != np.nan):
            rate = data.loc[i][line] / data.loc[i]['체결가']
        else:
            rate = np.nan
        rate_lst.append(rate)
    print(len(rate_lst))
    data['상승률'] = rate_lst
    return data

data=func1()
data=aftertimeprice(1,data)
data=data.fillna('missing')


data1=data
data1.index = data.시간
del data1['시간']
#data1=pd.DataFrame(data.체결가, index=data.시간)
print(data1)
df=pd.DataFrame()
df['상승률'] =data1.상승률.resample('1T').first()

print(df)

#excel 파일로 작성
"""base_dir="C:/Users/82107/PycharmProjects/DartProject"
file_nm="data.xlsx"
file_nm2="label.xlsx"
xlsx_dir=os.path.join(base_dir,file_nm)
data.to_excel(xlsx_dir)
xlsx_dir=os.path.join(base_dir,file_nm2)
data['상승률'].to_excel(xlsx_dir)"""